<a href="https://colab.research.google.com/github/cml33333/cml33333/blob/main/Nba_stat_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nba_api
#gets us data from nba

In [ ]:
from nba_api.stats.static import players #gets us players from nba
from nba_api.stats.endpoints import playergamelog, commonplayerinfo #gets us player game stats
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def get_player_id(player_name): #givena players name it will get us their id
  player_d = players.get_players()
  for p in player_d:
    if p['full_name'].lower() == player_name.lower():
      player = p
  return player['id'] if player else None

In [ ]:
def get_player_year(player_id): # uses panada function to get year started
  player_info = commonplayerinfo.CommonPlayerInfo(player_id = player_id)
  carrer_start_year = player_info.get_data_frames()[0]['FROM_YEAR'][0]
  return int(carrer_start_year)

In [ ]:
#interactive player selection
player_name = input("Enter Nba Player name: ")
player_id = get_player_id(player_name)
year = get_player_year(player_id)

Enter Nba Player name: Kyrie Irving


In [ ]:
if player_id is None:
  print("Player does not exist")
else:
  start_year = get_player_year(player_id)
  #Intializes empty data frame
  all_seasons_data = pd.DataFrame()
  #loop from start year to now to get data from everyseaon
  for year in range(start_year,2024):
    season_str = str(year) + '-' + str(year+1)[-2:]
    gamelog = playergamelog.PlayerGameLog(player_id = player_id,season = season_str)
    season_data = gamelog.get_data_frames()[0]
    all_seasons_data = pd.concat([all_seasons_data, season_data], ignore_index=True)
  all_seasons_data['OPPONENT_TEAM'] = all_seasons_data['MATCHUP'].apply(lambda x: x.split(' ')[-1])


In [ ]:
encoder = OneHotEncoder(sparse_output = False)
opponent_team_encoded = encoder.fit_transform(all_seasons_data[['OPPONENT_TEAM']])

opponent_team_df = pd.DataFrame(opponent_team_encoded, columns=encoder.get_feature_names_out(['OPPONENT_TEAM']))

In [ ]:
df_encoded = all_seasons_data.join(opponent_team_df)
feature_columns = [col for col in df_encoded.columns if 'OPPONENT_TEAM_' in col]

score_threshold = float(input("Input player points prop: "))
X = df_encoded[feature_columns]
Y = (df_encoded['PTS'] > score_threshold).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state = 42)
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, Y_train)

Input player points prop: 26.5


RandomForestClassifier()

In [ ]:
opponent_team = input("Team they are playing against: ")
input_data = pd.DataFrame(columns=feature_columns)
input_data.loc[0,:] = 0
input_data.loc[0, f'OPPONENT_TEAM_{opponent_team.upper()}']=1

Team they are playing against: PHI


In [ ]:
prediction = random_forest_model.predict(input_data)
prediction_propability = random_forest_model.predict_proba(input_data)

prediction_result = "Yes" if prediction[0] == 1 else "No"
probability = prediction_propability[0][prediction[0]]
print(f"Prediction: {prediction_result}, Probability: {probability*100}")

Prediction: No, Probability: 71.69369221278393
